# Analysis of: 
#### File originally retrieved from: 

<h2 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
FIRST PART: Pre-Analysis<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h3> 

## 1.1. Declaring the database dump file location 

In [ ]:
dumpfile='<filename>'

## 1.2. Taking a look on the raw dump file (first 100 lines)

In [ ]:
lines_to_read=100

with open(dumpfile) as myfile:
    firstlines=myfile.readlines()[0:lines_to_read] #put here the interval you want
    for x in firstlines:
        print(x.strip())

<h2 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
SECOND PART: Adapt to Our Generic Database Schema<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h3> 

## 2.0. Used Libraries

In [ ]:
import pandas as pd
import numpy as np

from io import StringIO
import re

from IPython.display import display,clear_output

## 2.1. Listing tables that have content inserted into the dump file

In [ ]:
def list_tables_with_insert(dumpfile):
    tables = []
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            if line.lower().startswith('insert'):
                table = re.findall(r'`(.*?)`', line)
                tables.append(table[0])
    return tables

In [ ]:
tables=enumerate(list_tables_with_insert(dumpfile))

for i, item in tables:
    print(i+1,item)

## 2.2. Read tables, one by one, and perform required adaptations 

In [ ]:
def read_inserted_table(dumpfile, target_table):
    sio = StringIO()
    fast_forward = True
    with open(dumpfile, 'rb') as f:
        for line in f:
            line = line.decode("utf-8").strip()
            line = re.sub("(?!(([^']*'){2})*[^']*$)\)", '',line) #Step0:remove any ")" from the content of columns
            if line.lower().startswith('insert') and target_table in line:
                fast_forward = False
            if fast_forward:
                continue
            data = re.findall('\([^\)]*\)', line) #Step1: get the content between parentesis (i.e., insert line)
            try:
                newline = data[0].strip('()') #Step2:remove parenthesis
                newline=newline.replace('`','') #Step3: remove ` (usually in table names)
                newline=re.sub(r'(?!(([^\']*\'){2})*[^\']*$),', '', newline) #Step4: remove commas from the content of columns
                newline=newline.replace('\'','') #Step5: remove single quotes
                newline=newline.replace(', ', ',') #Step6: remove single spaces after comma (i.e., in the beginning of a column)
                sio.write(newline)
                sio.write("\n")
            except IndexError:
                pass
            if line.endswith(';'):
                break
    sio.seek(0)
    return sio

## 2.2.0. Converter functions

In [ ]:
def tolowercase(text):
    try:
        return text.lower()
    except AttributeError:
        return text 

In [ ]:
def timestamp2datetime(timestamp):
    try:
        return  pd.to_datetime(timestamp,unit='s')
    except AttributeError:
        return timestamp 

## 2.2.1. Read the raw table: 'tablename'

In [ ]:
tablename='<tablename>'

pd.read_csv(read_inserted_table(dumpfile, tablename),
            delimiter=",",
            error_bad_lines=False).head(10)

#### Q1: Is this table different from other previous loaded table? YES|NO
#### Q2: Is this table similar to any table in the generic Booter database schema? YES|NO
    - This table looks like: 
    - The blacklist table must to have the columns: 
#### Q3: Are there modifications required? YES|NO
    - On the table name: 
        o =>
    - On the column type: 
        o =>
    - On the column name:
        o => 
    - Add required columns:
        o 

## 2.2.1.X Loading the modified table: 'tablename'

In [ ]:
df_<generictable> = pd.read_csv(read_inserted_table(dumpfile, tablename),
                                delimiter=",",
                                error_bad_lines=False,)
#                        converters = {'date':timestamp2datetime})

df_<generictable>.rename(columns = {'actual1': 'generic_column_name1',
                                    'actual2': 'generic_column_name2'},
                         inplace=True)

#Creating empty columns (with "" [for future string] or np.nan [for future float])
df_<generictable>['status'] = np.nan
df_<generictable>['lastchecked'] = np.nan
df_<generictable>['attacktype'] = np.nan

df_payments.head(10)

## 3.1. Adding missing accordingly to our generic Booter database 

In [ ]:
# df_attacks= pd.DataFrame(columns=['id','userid','targeturl','targetip','duration','port','date'])


<h1 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
THIRD PART: Data Enrichment<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h1> 

## 3.0. Needed Libraries

In [ ]:
import subprocess

import cfscrape
from lxml import etree

import os.path

import random
import time 

## 3.1. Lookup functions

In [ ]:
# THANKS TO: team-cymru.org
def iptoasn(iplist_teamcymruformat_filelocation,outputfile):
    cat = subprocess.Popen(['cat', iplist_teamcymruformat_filelocation], 
                            stdout=subprocess.PIPE)
    
    netcat = subprocess.Popen(['netcat', 'whois.cymru.com', '43'],
                              stdin=cat.stdout,
                              stdout=outputfile)
    time.sleep(3) #for some reason the poll does not work! This was the way to overcome the waiting time.
    
    return netcat.stdout      

In [ ]:
# THANKS TO: exonerator.torproject.org
def WasTorNode(ip, date ):
    url="https://exonerator.torproject.org/?ip="+ip+"&timestamp="+date
    scraper = cfscrape.create_scraper()
    scraped_html=scraper.get(url).content    
    html_tree = etree.HTML(scraped_html)
    result=html_tree.xpath("//h3[@class='panel-title']/text()") # I was looking for <h3 class="panel-title">Result is positive</h3>
    tor_node=True if result == ['Result is positive'] else False
    return tor_node 
# 'date' MUST BE formated as: Year-month-day (2016-03-31)

## 3.2. Discovering the middle date of the database

In [ ]:
try:
    try:
        middle_date=(min(df_attacks['date'])+((max(df_attacks['date'])-min(df_attacks['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_logins['date'])+((max(df_logins['date'])-min(df_logins['date']))/2))
        raise
    except Exception:
        pass

    try:
        middle_date=(min(df_payments['date'])+((max(df_payments['date'])-min(df_payments['date']))/2))
        raise
    except:
        pass
except Exception:
    print("There is no date in the entire dataset")

date_tor_check = middle_date.strftime('%Y-%m-%d')
date_iptoasn_lookup= str(middle_date)
print(date_tor_check)

## 3.3 Preparing to Perform IP to ASN info

In [ ]:
df_logins['middledate']=date_iptoasn_lookup
df_attacks['middledate']=date_iptoasn_lookup
df_friendsenemies['middledate']=date_iptoasn_lookup
df_blacklist['middledate']=date_iptoasn_lookup

## 3.4.1  Lookup IP to ASN info of table: logins

In [ ]:
if (os.path.exists('logins_iptoasn_out')== False):
    logins_iptoasn_in = open('logins_iptoasn_in', 'w+')
    logins_iptoasn_in.write('begin\nverbose\n')
    df_logins[['userip','middledate']].drop_duplicates().to_csv(logins_iptoasn_in,header=False,index=False,sep="\t") 
    logins_iptoasn_in.write('end')
    logins_iptoasn_in.close()

    logins_iptoasn_out = open('logins_iptoasn_out', 'w+')
    iptoasn('logins_iptoasn_in',logins_iptoasn_out)
    logins_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_logins_iptoasn = pd.read_csv('logins_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])
df_logins_iptoasn.head(10)

## 3.4.2  Lookup IP to ASN info of table: attacks

In [ ]:
if (os.path.exists('attacks_iptoasn_out')== False):
    attacks_iptoasn_in = open('attacks_iptoasn_in', 'w+')
    attacks_iptoasn_in.write('begin\nverbose\n')
    df_attacks[['targetip','middledate']].drop_duplicates().to_csv(attacks_iptoasn_in,header=False,index=False,sep="\t") 
    attacks_iptoasn_in.write('end')
    attacks_iptoasn_in.close()

    attacks_iptoasn_out = open('attacks_iptoasn_out', 'w+')
    iptoasn('attacks_iptoasn_in',attacks_iptoasn_out)
    attacks_iptoasn_out.close()
else:
    print("You already performed the lookup for this table")

In [ ]:
df_attacks_iptoasn = pd.read_csv('attacks_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])
df_attacks_iptoasn.head(10)

## 3.4.3  Lookup IP to ASN info of table: friendsenemies

In [ ]:
if (os.path.exists('friendsenemies_iptoasn_out')== False):
    friendsenemies_iptoasn_in = open('friendsenemies_iptoasn_in', 'w+')
    friendsenemies_iptoasn_in.write('begin\nverbose\n')
    df_friendsenemies[['ip','middledate']].drop_duplicates().to_csv(friendsenemies_iptoasn_in,header=False,index=False,sep="\t") 
    friendsenemies_iptoasn_in.write('end')
    friendsenemies_iptoasn_in.close()

    friendsenemies_iptoasn_out = open('friendsenemies_iptoasn_out', 'w+')
    iptoasn('friendsenemies_iptoasn_in',friendsenemies_iptoasn_out)
    friendsenemies_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_friendsenemies_iptoasn = pd.read_csv('logins_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])
df_friendsenemies_iptoasn.head(10)

## 3.4.4  Lookup IP to ASN info of table: blacklist

In [ ]:
if (os.path.exists('blacklist_iptoasn_out')== False):
    blacklist_iptoasn_in = open('blacklist_iptoasn_in', 'w+')
    blacklist_iptoasn_in.write('begin\nverbose\n')
    df_blacklist[['ip','middledate']].drop_duplicates().to_csv(blacklist_iptoasn_in,header=False,index=False,sep="\t") 
    blacklist_iptoasn_in.write('end')
    blacklist_iptoasn_in.close()

    blacklist_iptoasn_out = open('blacklist_iptoasn_out', 'w+')
    iptoasn('blacklist_iptoasn_in',blacklist_iptoasn_out)
    blacklist_iptoasn_out.close()
else:
    print("You already performed the lookup for this table.")

In [ ]:
df_blacklist_iptoasn = pd.read_csv('blacklist_iptoasn_out',\
                                skiprows=1,\
                             delimiter="\s+\|\s",\
                             names = ['asn', 'ip', 'bgp_prefix', 'country','registry','info_date','info_request','as_name'])
df_blacklist_iptoasn.head(10)

# 3.5.1. Check if IP was a TOR node for table: login

In [ ]:
date_tor_check

In [ ]:
if (os.path.exists('logins_torcheck')== False):
    print("Note: it can take a while to finish...")
    logins_torcheck = open('logins_torcheck', 'w+')
    for i in df_logins['userip'].unique():
        wasTor=WasTorNode(i,date_tor_check)
        print(i, wasTor, file=logins_torcheck)
#         print(i, wasTor) #DEBUGING =D
        time.sleep(random.randint(1,3)) #adding some random sleep time
        logins_torcheck.flush()

    logins_torcheck.close()
else:
    print("The IP addresses from this table were already checked.")

In [ ]:
df_logins_torcheck = pd.read_csv('logins_torcheck',\
                          delimiter="\s+",\
                          names = ['userip', 'tor'])

# 3.5.2. Check if IP was a TOR node for table: attacks

In [ ]:
if (os.path.exists('attacks_torcheck')== False):
    print("Note: it can take a while to finish...")

    attacks_torcheck = open('attacks_torcheck', 'w+')

    for i in df_attacks['targetip'].unique():
        wasTor=WasTorNode(i,date_tor_check)
        print(i, wasTor, file=attacks_torcheck)
    #     print(i, wasTor) #DEBUGING =D
        time.sleep(random.randint(1,3)) #adding some random sleep time
        attacks_torcheck.flush()

    attacks_torcheck.close()
else:
    print("The IP addresses from this table were already checked.")    

In [ ]:
df_attacks_torcheck = pd.read_csv('attacks_torcheck',\
                          delimiter="\s+",\
                          names = ['targetip', 'tor'])

# 3.5.3. Check if IP was a TOR node for table: friendsenemies

In [ ]:
if (os.path.exists('friendsenemies_torcheck')== False):
    print("Note: it can take a while to finish...")

    friendsenemies_torcheck = open('friendsenemies_torcheck', 'w+')

    for i in df_friendsenemies['ip'].unique():
        wasTor=WasTorNode(i,date_tor_check)
        print(i, wasTor, file=friendsenemies_torcheck)
    #     print(i, wasTor) #DEBUGING =D
        time.sleep(random.randint(1,3)) #adding some random sleep time
        friendsenemies_torcheck.flush()

    friendsenemies_torcheck.close()
else:
    print("The IP addresses from this table were already checked.") 

In [ ]:
df_friendsenemies_torcheck = pd.read_csv('friendsenemies_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

# 3.5.4. Check if IP was a TOR node for table: blacklist

In [ ]:
if (os.path.exists('blacklist_torcheck')== False):
    print("Note: it can take a while to finish...")
    
    blacklist_torcheck = open('blacklist_torcheck', 'w+')

    for i in df_blacklist['ip'].unique():
        wasTor=WasTorNode(i,date_tor_check)
        print(i, wasTor, file=blacklist_torcheck)
#         print(i, wasTor) #DEBUGING =D
        time.sleep(random.randint(1,3)) #adding some random sleep time
        blacklist_torcheck.flush()

    blacklist_torcheck.close()
else:
    print("The IP addresses from this table were already checked.")      

In [ ]:
df_blacklist_torcheck = pd.read_csv('blacklist_torcheck',\
                          delimiter="\s+",\
                          names = ['ip', 'tor'])

<h2 align="center">
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
FOURTH PART: Automatic Analysis
<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h3> 

## 5.0. Used Libraries

In [ ]:
import matplotlib.pyplot as plt
from matplotlib_venn import *
%matplotlib inline

plt.style.use('seaborn-muted')
plt.rcParams['font.family'] = 'serif'
plt.rcParams['font.size'] = 12

import warnings
warnings.filterwarnings('ignore')

## 5.1. Number of users, customers, attackers, and their intersection

In [ ]:
users_set=set(df_users['userid'].unique())
customers_set=set(df_payments['userid'].unique())
attackers_set=set(df_attacks['userid'].unique())

intersec_customers_attacker=pd.Series(list(customers_set.intersection(attackers_set)))
intersec_users_customers=pd.Series(list(users_set.intersection(customers_set)))
intersec_users_attackers=pd.Series(list(users_set.intersection(attackers_set)))

intersec_users_customers_attackers=pd.Series(list(users_set.intersection(customers_set).intersection(attackers_set)))

In [ ]:
fig = plt.figure(figsize=(4,3))
fig.suptitle('Users, Customers & Attackers', fontsize=16)

ax = plt.subplot2grid((1,1), (0,0))

venn=venn3(ax=ax,subsets = {'001':len(attackers_set)-len(intersec_customers_attacker)-len(intersec_users_attackers)+len(intersec_users_customers_attackers), 
                            '010':len(customers_set)-len(intersec_users_customers)-len(intersec_customers_attacker)+len(intersec_users_customers_attackers), 
                            '011':len(intersec_customers_attacker)-len(intersec_users_customers_attackers),
                            '100':len(users_set)-len(intersec_users_customers)-len(intersec_users_attackers)+len(intersec_users_customers_attackers),
                            '101':len(intersec_users_attackers)-len(intersec_users_customers_attackers),
                            '110':len(intersec_users_customers)-len(intersec_users_customers_attackers),
                            '111':len(intersec_users_customers_attackers)},\
          set_labels = ('Users', 'Customers','Attackers'),\
          alpha=1)
try:
    venn.get_patch_by_id('100').set_color('#404096')
except:
    print("*Users set is empty!")  
    
try:
    venn.get_patch_by_id('110').set_color('#DEA73A')
except:
    print("*Customers set is empty!")   

try:
    venn.get_patch_by_id('001').set_color('#D92120')
except:
    print("*Attackers set is empty!")

fig.show()
# fig.savefig('../figs/timeseries_attacks.eps', format='eps', dpi=1200) 

## 5.2. Distribution of login times per user

In [ ]:
num_distinct_logins_per_user=df_logins['userid'].value_counts()
freq_distinct_logins_per_user=num_distinct_logins_per_user.value_counts()
cum_dist_user_logins = np.linspace(0.,1.,len(num_distinct_logins_per_user))
cdf_user_logins = pd.Series(cum_dist_user_logins, index=num_distinct_logins_per_user.sort_values())

In [ ]:
if len(num_distinct_logins_per_user)>0:
    fig = plt.figure()
    fig = plt.figure(figsize=(7, 3))
    fig.suptitle('Distribution of Login Times by Users', fontsize=16)

    #Plot CDF
    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_logins.plot(ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# logins")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_logins_per_user.plot(ax=ax2,kind='pie', autopct='%1.1f%%', startangle=270, fontsize=10)
    ax2.set_ylabel("")

    fig.show()
else:
    print("Unfortunately, there is no data available!")
# fig.savefig('../figs/cdf_users_attacks.eps', bbox_inches='tight',format='eps', dpi=1200)

## 5.3. Distribution of IP addresses per user

In [ ]:
num_distinct_ips_per_user=df_logins.groupby(['userid','userip']).size().reset_index()['userid'].value_counts()
freq_distinct_ips_per_user=num_distinct_ips_per_user.value_counts()
cum_dist_user_ips = np.linspace(0.,1.,len(num_distinct_ips_per_user))
cdf_user_ips = pd.Series(cum_dist_user_ips, index=num_distinct_ips_per_user.sort_values())

In [ ]:
if len(num_distinct_ips_per_user)>0:
    fig = plt.figure()
    fig = plt.figure(figsize=(7, 3))
    fig.suptitle('Distribution of Distinct IP address used by Users', fontsize=16)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_ips.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# IPs")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_ips_per_user.plot(ax=ax2,kind='pie', autopct='%1.1f%%', startangle=270, fontsize=10)
    ax2.set_ylabel("")

    fig.show()
else:
    print("Unfortunately, there is no data available!")
# fig.savefig('../figs/cdf_users_attacks.eps', bbox_inches='tight',format='eps', dpi=1200)

## 5.4. Distribution of Number of Payments by Users

In [ ]:
num_distinct_payments_per_user=df_payments['userid'].value_counts()
freq_distinct_payments_per_user=num_distinct_payments_per_user.value_counts()
cum_dist_user_payments = np.linspace(0.,1.,len(num_distinct_payments_per_user))
cdf_user_payments = pd.Series(cum_dist_user_payments, index=num_distinct_payments_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_per_user)>0:
    fig = plt.figure()
    fig = plt.figure(figsize=(7, 3))
    fig.suptitle('Distribution of Number of Payments by Users', fontsize=16)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("# Payment")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_per_user.plot(ax=ax2,kind='pie', autopct='%1.1f%%', startangle=270, fontsize=10)
    ax2.set_ylabel("")

    fig.show()
else:
    print("Unfortunately, there is no data available!")
# fig.savefig('../figs/cdf_users_attacks.eps', bbox_inches='tight',format='eps', dpi=1200)

## 5.5. Total amount of money earned

In [ ]:
total_earned=df_payments['amountpaid'].values.sum()
'US$ {:,.2f}'.format(float(total_earned))

## 5.6. Distribution of the money paid by Users

In [ ]:
num_distinct_payments_money_per_user=df_payments['amountpaid'].value_counts()
freq_distinct_payments_money_per_user=num_distinct_payments_money_per_user.value_counts()
cum_dist_user_payments_money = np.linspace(0.,1.,len(num_distinct_payments_money_per_user))
cdf_user_payments_money = pd.Series(cum_dist_user_payments_money, index=num_distinct_payments_money_per_user.sort_values())

In [ ]:
if len(num_distinct_payments_money_per_user)>0:
    fig = plt.figure()
    fig = plt.figure(figsize=(7, 3))
    fig.suptitle('Distribution of Money Payments by Users', fontsize=16)

    ax1 = plt.subplot2grid((1,2), (0,0))
    ax1 = cdf_user_payments_money.plot( ax=ax1,lw=2, drawstyle='steps',legend=False)
    ax1.set_xlabel("US$")
    ax1.set_ylabel("CDF of Users")
    ax1.set_ylim(0,1.05)

    ax2 = plt.subplot2grid((1,2), (0,1))
    freq_distinct_payments_money_per_user.plot(ax=ax2,kind='pie', autopct='%1.1f%%', startangle=270, fontsize=10)
    ax2.set_ylabel("")

    fig.show()
else:
    print("Unfortunately, there is no data available!")
# fig.savefig('../figs/cdf_users_attacks.eps', bbox_inches='tight',format='eps', dpi=1200)

## 5.7. Distribution of Countries that users access Booters

In [ ]:
logins_country_distribution=df_logins_iptoasn['country'].value_counts()

In [ ]:
if len(logins_country_distribution)>0:
    fig = plt.figure()
    fig = plt.figure(figsize=(7, 3))
    fig.suptitle('Users Accessing from Countries', fontsize=16)

    ax1 = plt.subplot2grid((1,2), (0,0))
    logins_country_distribution.plot(ax=ax1,kind='bar')
    ax1.set_ylabel("")

    ax2 = plt.subplot2grid((1,2), (0,1))
    logins_country_distribution.plot(ax=ax2,kind='pie', autopct='%1.1f%%', startangle=270, fontsize=10)
    ax2.set_ylabel("")

    fig.show()
else:
    print("Unfortunately, there is no data available!")

## 5.8. Countries of blacklisted IPs

In [ ]:
blacklist_country_distribution=df_blacklist_iptoasn['country'].value_counts()

In [ ]:
if len(blacklist_country_distribution)>0:
    fig = plt.figure()
    fig = plt.figure(figsize=(7, 3))
    fig.suptitle('Countries of blacklisted IPs', fontsize=16)

    ax1 = plt.subplot2grid((1,2), (0,0))
    blacklist_country_distribution.plot(ax=ax1,kind='bar')
    ax1.set_ylabel("")

    ax2 = plt.subplot2grid((1,2), (0,1))
    blacklist_country_distribution.plot(ax=ax2,kind='pie', autopct='%1.1f%%', startangle=270, fontsize=10)
    ax2.set_ylabel("")

    fig.show()
else:
    print("Unfortunately, there is no data available!")

## 5.9. Countries of target IPs

In [ ]:
attacks_country_distribution=df_attacks_iptoasn['country'].value_counts()

In [ ]:
if len(attacks_country_distribution)>0:
    fig = plt.figure()
    fig = plt.figure(figsize=(7, 3))
    fig.suptitle('Countries of target IPs', fontsize=16)

    ax1 = plt.subplot2grid((1,2), (0,0))
    attacks_country_distribution.plot(ax=ax1,kind='bar')
    ax1.set_ylabel("")

    ax2 = plt.subplot2grid((1,2), (0,1))
    attacks_country_distribution.plot(ax=ax2,kind='pie', autopct='%1.1f%%', startangle=270, fontsize=10)
    ax2.set_ylabel("")

    fig.show()
else:
    print("Unfortunately, there is no data available!")

<h2 align="center">!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!<br>
FIFTH PART: Query Interface<br>
!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!</h2> 

In [ ]:
from ipywidgets import widgets, interact
from IPython.display import display,clear_output

##  5.1. Search user by name or email

In [ ]:
def searchfor_username_submit(sender):
    clear_output()
    print("Searching for \"",searchfor_username.value,"\" in df_users:")
    display(df_users[df_users['username']== searchfor_username.value.lower()])

def searchfor_useremail_submit(sender):
    clear_output()
    print("Searching for \"",searchfor_useremail.value,"\" in df_users:")
    display(df_users[df_users['useremail']== searchfor_useremail.value])
    print("Searching for \"",searchfor_useremail.value,"\" in df_payment:")
    display(df_payments[df_payments['paymentemail']== searchfor_useremail.value])

In [ ]:
searchfor_username = widgets.Text(description="username")
display(searchfor_username)
searchfor_username.on_submit(searchfor_username_submit)

searchfor_useremail = widgets.Text(description="email")
display(searchfor_useremail)
searchfor_useremail.on_submit(searchfor_useremail_submit)

## 5.2. Search attacks by username or userid

In [ ]:
def searchfor_attacks_userid_submit(sender):
    clear_output()
    print("Searching for \"",searchfor_attacks_userid.value,"\" in df_attacks:")
    display(df_attacks[df_attacks['userid']== searchfor_attacks_userid.value])

def searchfor_attacks_username_submit(sender):
    clear_output()
    print("Searching for \"",searchfor_attacks_username.value,"\" in df_attacks:")
    userid=df_users[df_users['username']==searchfor_attacks_username.value.lower()]['userid'].iloc[0]
    display(df_attacks[df_attacks['userid']== userid])

In [ ]:
searchfor_attacks_username = widgets.Text(description="username",)
display(searchfor_attacks_username)
searchfor_attacks_username.on_submit(searchfor_attacks_username_submit)

searchfor_attacks_userid = widgets.Text(description="userid")
display(searchfor_attacks_userid)
searchfor_attacks_userid.on_submit(searchfor_attacks_userid_submit)

## 5.3. Search IP address by userid or username

In [ ]:
def search_ips_by_userid_submit(sender):
    clear_output()
    print("Searching for \"",search_ips_by_userid.value,"\" in df_logins:")
    display(df_logins[df_logins['userid']== int(search_ips_by_userid.value)])

def search_ips_by_username_submit(sender):
    clear_output()
    print("Searching for \"",search_ips_by_username.value,"\" in df_logins:")
    userid=df_users[df_users['username']==search_ips_by_username.value.lower()]['userid'].iloc[0]
    display(df_logins[df_logins['userid']== userid])

In [ ]:
search_ips_by_username = widgets.Text(description="username")
display(search_ips_by_username)
search_ips_by_username.on_submit(search_ips_by_username_submit)

search_ips_by_userid = widgets.Text(description="userid")
display(search_ips_by_userid)
search_ips_by_userid.on_submit(search_ips_by_userid_submit)